Website Application

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import pickle
from prepare_data import make_predictions

# Load the model
#with open('./artifacts/model_2.pkl', 'rb') as f:
    #model = pickle.load(f)

#Initialise the Dash App
app = dash.Dash(__name__)
server = app.server

#Define App Layout
app.layout = html.Div([
    html.H1("Churn Eligibility Predictor"),
    html.Label("CustomerID:"),
    dcc.Input(id='CustomerID', type='number', value=0),
    html.Label("Gender:"),
    dcc.Dropdown(
        ['Male','Female'],
        'Male',
        id='Gender'
    ),
    html.Label("Age:"),
    dcc.Slider(0, 100, 1, marks=None, value=20, id='Age'),
    html.Label("Income:"),
    dcc.Slider(0, 100000, 1000, marks=None, value=10000, id='Income'),
    html.Label("TotalPurchase:"),
    dcc.Slider(0, 100000, 100, marks=None, value=1000, id='TotalPurchase'),
    html.Label("NumOfPurchases:"),
    dcc.Input(id='NumOfPurchases', type='number', value=0),
    html.Label("Location:"),
    dcc.Dropdown(
        ['Urban','Suburban','Rural'], 
        'Urban',
        id='Location'
    ),
    html.Label("MaritalStatus:"),
    dcc.Dropdown(
        ['Married','Single'],
        'Married',
        id='MaritalStatus'
    ),
    html.Label("Education:"),
    dcc.Dropdown(
        ['High School',"Bachelor's","Master's",'PhD'],
        'High School',
        id='Education'
    ),
    html.Label("Subscription Plan:"),
    dcc.Dropdown(
        ['Gold','Bronze','Silver'],
        'Bronze',
        id='SubscriptionPlan'
    ),
    html.Button('Check Eligibility', id='submit-val', n_clicks=0),
    html.Div(id='output')
])
# Define Callback Function for Predictions
@app.callback(
    Output('output', 'children'),
    Input('submit-val', 'n_clicks'),
    Input('CustomerID','value'),
    Input('Gender', 'value'),
    Input('Age', 'value'),
    Input('Income', 'value'),
    Input('TotalPurchase', 'value'),
    Input('NumOfPurchases', 'value'),
    Input('Location', 'value'),
    Input('MaritalStatus', 'value'),
    Input('Education', 'value'),
    Input('SubscriptionPlan', 'value')
)
def update_output(n_clicks, CustomerID, Gender, Age, Income, TotalPurchase, NumOfPurchases,
                    Location, MaritalStatus, Education, SubscriptionPlan):
    if n_clicks > 0:
        # Prepare input data for prediction
        input_data = pd.DataFrame({
            'CustomerID': [CustomerID],
            'Gender': [Gender],
            'Age': [Age],
            'Income': [Income],
            'TotalPurchase': [TotalPurchase],
            'NumOfPurchases': [NumOfPurchases],
            'Location': [Location],
            'MaritalStatus': [MaritalStatus],
            'Education': [Education],
            'SubscriptionPlan': [SubscriptionPlan]
        })
        # One-hot encode categorical variables
        #input_data = pd.get_dummies(input_data)
        #prepare the input dataframe
        #input_data = prepare_data(input_data)
        # Make predictions
        prediction = make_predictions(input_data)
        if prediction[0] == 1:
            return 'Churn Approved'#, style={'color': 'green'})
        else:
            return 'Churn Rejected'#, style={'color': 'red'})
#Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\micha\AppData\Local\Temp\ipykernel_27432\131023205.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\micha\AppData\Local\Temp\ipykernel_27432\131023205.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


: 